In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mtick
from matplotlib.pyplot import figure
import scipy as sp
import calendar
import seaborn as sns
import os
from scipy.stats import pearsonr
from stargazer.stargazer import Stargazer
import statsmodels.formula.api as smf
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [2]:
!pip install tabulate

You should consider upgrading via the 'c:\users\19084\anaconda3\python.exe -m pip install --upgrade pip' command.


## Skip to monthly data set if you are focusing on prediction

## Digital Fundametals

In [3]:
factor_dict=pd.read_excel('C:/Users/19804/My Backup Files/Digital Assets/dict.xlsx')
factor_dict.head()

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/19804/My Backup Files/Digital Assets/dict.xlsx'

# Pre-Process Data Functions

In [4]:
def factor_read(curr):
    wkdir = 'C:/Users/Karen/My Backup Files/Digital Assets/'
    factor_data = wkdir+'CoinMetricsData/' + curr + '.csv'
    dfname = 'factor_data'
    dfname = pd.read_csv(factor_data)
    return dfname

In [5]:
def factor_df(df_in,curr):
    df_out=pd.DataFrame(columns=['date'])
    df_out['date']=df_in['date']   
    for col in df_in.columns:
        for j in range(0,len(factor_dict)):
            if col==factor_dict['Factor'].iloc[j]:
                new_col=curr + '_' + factor_dict['FactorNum'].iloc[j]
                df_out[new_col]=df_in[col]
    return df_out           

## Read and process data

In [6]:
btc_factor_raw=factor_read('BTC')
btc_factor_raw.head()

,date,AdrActCnt,BlkCnt,BlkSizeByte,BlkSizeMeanByte,CapMVRVCur,CapMrktCurUSD,CapRealUSD,DiffMean,FeeMeanNtv,...,TxTfrValAdjUSD,TxTfrValMeanNtv,TxTfrValMeanUSD,TxTfrValMedNtv,TxTfrValMedUSD,TxTfrValNtv,TxTfrValUSD,VtyDayRet180d,VtyDayRet30d,VtyDayRet60d
0,2009-01-03,0,0,0,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1,2009-01-04,0,0,0,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
2,2009-01-05,0,0,0,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
3,2009-01-06,0,0,0,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
4,2009-01-07,0,0,0,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [7]:
btc_factor_df=factor_df(btc_factor_raw,'BTC')
btc_factor_df.head()

,date,BTC_F1,BTC_F2,BTC_F3,BTC_F4,BTC_F5,BTC_F6,BTC_F7,BTC_F8,BTC_F9,...,BTC_F22,BTC_F23,BTC_F24,BTC_F25,BTC_F26,BTC_F27,BTC_F28,BTC_F29,BTC_F30,BTC_F31
0,2009-01-03,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
1,2009-01-04,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
2,2009-01-05,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
3,2009-01-06,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
4,2009-01-07,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN


In [8]:
eth_factor_raw=factor_read('ETH')
eth_factor_raw.head()

,date,AdrActCnt,BlkCnt,BlkSizeByte,BlkSizeMeanByte,CapMrktCurUSD,DiffMean,FeeMeanNtv,FeeMeanUSD,FeeMedNtv,...,TxTfrValAdjUSD,TxTfrValMeanNtv,TxTfrValMeanUSD,TxTfrValMedNtv,TxTfrValMedUSD,TxTfrValNtv,TxTfrValUSD,VtyDayRet180d,VtyDayRet30d,VtyDayRet60d
0,2015-07-30,9206,6911,4449897,643.886124,NaN,1.213355e+11,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1,2015-07-31,424,6863,3994458,582.027976,NaN,6.029419e+11,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
2,2015-08-01,413,5293,3044344,575.164179,NaN,8.872542e+11,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
3,2015-08-02,432,5358,3112348,580.878686,NaN,1.020254e+12,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
4,2015-08-03,444,5280,3099953,587.112311,NaN,1.125837e+12,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [9]:
eth_factor_df=factor_df(eth_factor_raw,'ETH')
eth_factor_df.head()

,date,ETH_F1,ETH_F2,ETH_F3,ETH_F4,ETH_F5,ETH_F6,ETH_F7,ETH_F8,ETH_F9,...,ETH_F22,ETH_F23,ETH_F24,ETH_F25,ETH_F26,ETH_F27,ETH_F28,ETH_F29,ETH_F30,ETH_F31
0,2015-07-30,9206,6911,4449897,643.886124,NaN,1.213355e+11,NaN,NaN,NaN,...,0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
1,2015-07-31,424,6863,3994458,582.027976,NaN,6.029419e+11,NaN,NaN,NaN,...,0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
2,2015-08-01,413,5293,3044344,575.164179,NaN,8.872542e+11,NaN,NaN,NaN,...,0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
3,2015-08-02,432,5358,3112348,580.878686,NaN,1.020254e+12,NaN,NaN,NaN,...,0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
4,2015-08-03,444,5280,3099953,587.112311,NaN,1.125837e+12,NaN,NaN,NaN,...,0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN


In [10]:
xrp_factor_raw=factor_read('XRP')
xrp_factor_raw.head()

,date,AdrActCnt,BlkCnt,CapMrktCurUSD,FeeMeanNtv,FeeMeanUSD,FeeMedNtv,FeeMedUSD,FeeTotNtv,FeeTotUSD,...,TxTfrValAdjUSD,TxTfrValMeanNtv,TxTfrValMeanUSD,TxTfrValMedNtv,TxTfrValMedUSD,TxTfrValNtv,TxTfrValUSD,VtyDayRet180d,VtyDayRet30d,VtyDayRet60d
0,2013-01-01,0,4164,NaN,NaN,NaN,NaN,NaN,0.00000,NaN,...,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN,NaN
1,2013-01-02,10,4703,NaN,0.00001,NaN,0.00001,NaN,0.00018,NaN,...,NaN,3.353333e+06,NaN,50000.0,NaN,1.006000e+07,NaN,NaN,NaN,NaN
2,2013-01-03,8,4780,NaN,0.00001,NaN,0.00001,NaN,0.00010,NaN,...,NaN,6.883333e+03,NaN,10000.0,NaN,4.130000e+04,NaN,NaN,NaN,NaN
3,2013-01-04,9,4024,NaN,0.00001,NaN,0.00001,NaN,0.00019,NaN,...,NaN,1.168539e+05,NaN,10000.0,NaN,7.011231e+05,NaN,NaN,NaN,NaN
4,2013-01-05,12,3201,NaN,0.00001,NaN,0.00001,NaN,0.00025,NaN,...,NaN,2.140400e+03,NaN,200.0,NaN,1.070200e+04,NaN,NaN,NaN,NaN


In [11]:
xrp_factor_df=factor_df(xrp_factor_raw,'XRP')
xrp_factor_df.head()

,date,XRP_F1,XRP_F2,XRP_F5,XRP_F7,XRP_F8,XRP_F9,XRP_F10,XRP_F11,XRP_F12,...,XRP_F22,XRP_F23,XRP_F24,XRP_F25,XRP_F26,XRP_F27,XRP_F28,XRP_F29,XRP_F30,XRP_F31
0,2013-01-01,0,4164,NaN,NaN,NaN,NaN,NaN,0.00000,NaN,...,0,0.000000e+00,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN
1,2013-01-02,10,4703,NaN,0.00001,NaN,0.00001,NaN,0.00018,NaN,...,3,1.006000e+07,NaN,3.353333e+06,NaN,50000.0,NaN,1.006000e+07,NaN,NaN
2,2013-01-03,8,4780,NaN,0.00001,NaN,0.00001,NaN,0.00010,NaN,...,6,4.130000e+04,NaN,6.883333e+03,NaN,10000.0,NaN,4.130000e+04,NaN,NaN
3,2013-01-04,9,4024,NaN,0.00001,NaN,0.00001,NaN,0.00019,NaN,...,6,7.011231e+05,NaN,1.168539e+05,NaN,10000.0,NaN,7.011231e+05,NaN,NaN
4,2013-01-05,12,3201,NaN,0.00001,NaN,0.00001,NaN,0.00025,NaN,...,5,1.070200e+04,NaN,2.140400e+03,NaN,200.0,NaN,1.070200e+04,NaN,NaN


## Moving average

In [12]:
def factor_ma(df_in,ma):
    df_out=pd.DataFrame(columns=['date'])
    df_out['date']=df_in['date']
    for col in df_in.columns:
        col_name_new=col + '_MA' + str(ma)
        if col!='date':
            df_out[col_name_new]=df_in[col].rolling(ma).mean()
    return df_out

In [13]:
btc_factor_ma22=factor_ma(btc_factor_df,22)
eth_factor_ma22=factor_ma(eth_factor_df,22)
xrp_factor_ma22=factor_ma(xrp_factor_df,22)

## Merge BTC-ETH

In [14]:
btc_eth_factor_ma_df = pd.merge(btc_factor_ma22,eth_factor_ma22, on='date', how='outer')
btc_eth_factor_ma_clean_df=btc_eth_factor_ma_df.dropna().copy()
btc_eth_factor_ma_clean_df.head()

,date,BTC_F1_MA22,BTC_F2_MA22,BTC_F3_MA22,BTC_F4_MA22,BTC_F5_MA22,BTC_F6_MA22,BTC_F7_MA22,BTC_F8_MA22,BTC_F9_MA22,...,ETH_F22_MA22,ETH_F23_MA22,ETH_F24_MA22,ETH_F25_MA22,ETH_F26_MA22,ETH_F27_MA22,ETH_F28_MA22,ETH_F29_MA22,ETH_F30_MA22,ETH_F31_MA22
2459,2015-09-28,311570.136364,147.272727,8.553845e+07,579881.074438,3.427570e+09,5.816676e+10,0.000194,0.045570,0.000097,...,5794.227273,342072.019783,320361.607002,72.713232,67.503565,1.979992,1.861693,411949.872077,380664.473275,0.109245
2460,2015-09-29,313265.454545,148.272727,8.592031e+07,579318.892692,3.426264e+09,5.827484e+10,0.000192,0.045094,0.000097,...,5800.181818,351598.570845,322969.339912,74.304871,67.594265,1.977662,1.803736,421141.206182,381455.449155,0.108324
2461,2015-09-30,313073.136364,148.818182,8.506403e+07,570940.558828,3.422213e+09,5.838292e+10,0.000192,0.044887,0.000097,...,5809.818182,360114.289817,323467.055120,76.520689,67.992275,1.973719,1.741110,434409.296802,384333.190925,0.107434
2462,2015-10-01,313009.590909,148.818182,8.323488e+07,558664.414838,3.422773e+09,5.852346e+10,0.000192,0.044851,0.000097,...,5848.227273,329677.722595,275454.752148,71.199989,59.260693,1.974043,1.690375,408470.719176,338323.931448,0.106533
2463,2015-10-02,314204.090909,147.318182,8.126928e+07,551839.460287,3.423109e+09,5.869871e+10,0.000190,0.044459,0.000097,...,5880.454545,328639.672441,271482.521016,70.552010,57.899324,1.955658,1.621210,405880.382187,331722.672623,0.105877


## Merge BTC-ETH_XRP

In [15]:
btc_eth_xrp_factor_ma_df = pd.merge(btc_eth_factor_ma_clean_df,xrp_factor_ma22, on='date', how='outer')
btc_eth_xrp_factor_ma_clean_df=btc_eth_xrp_factor_ma_df.dropna().copy()
btc_eth_xrp_factor_ma_clean_df.head()

,date,BTC_F1_MA22,BTC_F2_MA22,BTC_F3_MA22,BTC_F4_MA22,BTC_F5_MA22,BTC_F6_MA22,BTC_F7_MA22,BTC_F8_MA22,BTC_F9_MA22,...,XRP_F22_MA22,XRP_F23_MA22,XRP_F24_MA22,XRP_F25_MA22,XRP_F26_MA22,XRP_F27_MA22,XRP_F28_MA22,XRP_F29_MA22,XRP_F30_MA22,XRP_F31_MA22
0,2015-09-28,311570.136364,147.272727,8.553845e+07,579881.074438,3.427570e+09,5.816676e+10,0.000194,0.045570,0.000097,...,12125.363636,9.714054e+07,671581.083169,10440.827549,71.539504,918.181818,6.419069,1.031309e+08,713057.813676,0.028882
1,2015-09-29,313265.454545,148.272727,8.592031e+07,579318.892692,3.426264e+09,5.827484e+10,0.000192,0.045094,0.000097,...,11892.545455,1.070422e+08,716854.894735,11389.509215,76.061943,1018.181818,6.841585,1.129652e+08,757789.515521,0.028941
2,2015-09-30,313073.136364,148.818182,8.506403e+07,570940.558828,3.422213e+09,5.838292e+10,0.000192,0.044887,0.000097,...,11534.500000,1.116494e+08,738682.570134,11886.352777,78.630943,1100.000000,7.250696,1.175701e+08,779574.730815,0.029242
3,2015-10-01,313009.590909,148.818182,8.323488e+07,558664.414838,3.422773e+09,5.852346e+10,0.000192,0.044851,0.000097,...,10972.000000,1.174636e+08,762627.439192,13674.336716,87.930596,1095.454545,7.168383,1.279726e+08,828361.966980,0.029534
4,2015-10-02,314204.090909,147.318182,8.126928e+07,551839.460287,3.423109e+09,5.869871e+10,0.000190,0.044459,0.000097,...,10231.409091,1.228092e+08,790815.362052,15619.026498,99.160103,1195.454545,7.707435,1.332857e+08,856302.530702,0.029975


# Prices

In [16]:
def price_returns(curr):
    wkdir = 'C:/Users/Karen/My Backup Files/Digital Assets/'
    price_data = wkdir+'CoinMetricsData/' + curr + '.csv'
    temp = pd.read_csv(price_data)
    curr_price = curr+'_USD'
    df_out=pd.DataFrame(columns=['date',curr_price])
    df_out['date']=temp['date']
    df_out[curr_price]=temp['PriceUSD']
    df_out.dropna(inplace=True)
    df_out['month']= pd.DatetimeIndex(df_out['date']).month
    df_out['lmonth']=df_out['month'].shift(-1)
    df_out['diffmonth']=df_out['month']-df_out['lmonth']
    df_out['dmonth']=df_out['diffmonth'].apply(lambda x: 1 if x!=0 else 0)
    curr_eom = curr_price + '_EoM'
    df_out[curr_eom]=df_out['dmonth']*df_out[curr_price]
    eom_df=df_out[df_out[curr_eom]>0]
    curr_ret = curr + '_Ret'
    eom_df[curr_ret]=eom_df[curr_eom].pct_change()
    eom_df_clean=eom_df[['date',curr_price,curr_ret]].copy()
    return eom_df_clean
    # return df_out

In [17]:
btc_price=price_returns('BTC')
eth_price=price_returns('ETH')
xrp_price=price_returns('XRP')

<ipython-input-16-48cce05dcbb1>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eom_df[curr_ret]=eom_df[curr_eom].pct_change()


## Merge BTC-ETH prices and returns

In [18]:
btc_eth_price=pd.merge(btc_price,eth_price, on='date', how='outer')
btc_eth_price_clean=btc_eth_price.dropna()
btc_eth_price_clean['BTC_ETH_Ret']=btc_eth_price_clean['BTC_Ret']-btc_eth_price_clean['ETH_Ret']
btc_eth_price_clean.head()

<ipython-input-18-6fef5a56247a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  btc_eth_price_clean['BTC_ETH_Ret']=btc_eth_price_clean['BTC_Ret']-btc_eth_price_clean['ETH_Ret']


,date,BTC_USD,BTC_Ret,ETH_USD,ETH_Ret,BTC_ETH_Ret
62,2015-09-30,236.739848,0.026227,0.713644,-0.453778,0.480006
63,2015-10-31,314.119508,0.326855,0.889343,0.246199,0.080657
64,2015-11-30,377.389840,0.201421,0.880000,-0.010505,0.211926
65,2015-12-31,429.677153,0.138550,0.948840,0.078227,0.060323
66,2016-01-31,366.341688,-0.147402,2.242672,1.363593,-1.510996


In [19]:
btc_eth_xrp_price=pd.merge(btc_eth_price_clean,xrp_price, on='date', how='outer')
btc_eth_xrp_price_clean=btc_eth_xrp_price.dropna()
btc_eth_xrp_price_clean['BTC_XRP_Ret']=btc_eth_xrp_price_clean['BTC_Ret']-btc_eth_xrp_price_clean['XRP_Ret']
btc_eth_xrp_price_clean.head()

<ipython-input-19-67a591f02004>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  btc_eth_xrp_price_clean['BTC_XRP_Ret']=btc_eth_xrp_price_clean['BTC_Ret']-btc_eth_xrp_price_clean['XRP_Ret']


,date,BTC_USD,BTC_Ret,ETH_USD,ETH_Ret,BTC_ETH_Ret,XRP_USD,XRP_Ret,BTC_XRP_Ret
0,2015-09-30,236.739848,0.026227,0.713644,-0.453778,0.480006,0.005602,-0.271297,0.297524
1,2015-10-31,314.119508,0.326855,0.889343,0.246199,0.080657,0.004631,-0.173314,0.500169
2,2015-11-30,377.389840,0.201421,0.880000,-0.010505,0.211926,0.004204,-0.092315,0.293737
3,2015-12-31,429.677153,0.138550,0.948840,0.078227,0.060323,0.005955,0.416621,-0.278071
4,2016-01-31,366.341688,-0.147402,2.242672,1.363593,-1.510996,0.006358,0.067630,-0.215032


## Compute and model returns

In [20]:
btc_eth_xrp_model=pd.merge(btc_eth_xrp_price_clean,btc_eth_xrp_factor_ma_clean_df, on='date', how='outer')
btc_eth_xrp_model_keep=btc_eth_xrp_model.dropna()
btc_eth_xrp_model=btc_eth_xrp_model_keep
btc_eth_xrp_model_keep.head()

,date,BTC_USD,BTC_Ret,ETH_USD,ETH_Ret,BTC_ETH_Ret,XRP_USD,XRP_Ret,BTC_XRP_Ret,BTC_F1_MA22,...,XRP_F22_MA22,XRP_F23_MA22,XRP_F24_MA22,XRP_F25_MA22,XRP_F26_MA22,XRP_F27_MA22,XRP_F28_MA22,XRP_F29_MA22,XRP_F30_MA22,XRP_F31_MA22
0,2015-09-30,236.739848,0.026227,0.713644,-0.453778,0.480006,0.005602,-0.271297,0.297524,313073.136364,...,11534.500000,1.116494e+08,7.386826e+05,11886.352777,78.630943,1100.000000,7.250696,1.175701e+08,7.795747e+05,0.029242
1,2015-10-31,314.119508,0.326855,0.889343,0.246199,0.080657,0.004631,-0.173314,0.500169,350885.363636,...,7400.954545,1.151984e+08,5.508404e+05,24708.658755,122.547418,775.609818,3.740480,1.207873e+08,5.763011e+05,0.051605
2,2015-11-30,377.389840,0.201421,0.880000,-0.010505,0.211926,0.004204,-0.092315,0.293737,383127.136364,...,7896.590909,5.017505e+08,2.143587e+06,56445.962706,241.181455,1175.003091,5.039077,5.020341e+08,2.144797e+06,0.039679
3,2015-12-31,429.677153,0.138550,0.948840,0.078227,0.060323,0.005955,0.416621,-0.278071,456927.772727,...,14032.409091,3.548443e+08,2.827085e+06,21641.875393,151.513749,1255.822273,8.226846,3.555645e+08,2.832146e+06,0.080208
4,2016-01-31,366.341688,-0.147402,2.242672,1.363593,-1.510996,0.006358,0.067630,-0.215032,488123.454545,...,15790.000000,1.871757e+08,1.093580e+06,17802.197137,97.419745,1106.819091,5.986345,1.883054e+08,1.099983e+06,0.042615


## Lag the factors

In [22]:
btc_eth_xrp_model_keep.to_csv('btc_eth_xrp_model.csv',index=False)
btc_eth_xrp_model_keep.to_excel('btc_eth_xrp_model.xlsx',index=False)

In [23]:
def relative_factors(df_in,first_col,curr1,curr2):
    df_out=df_in.iloc[:,0:first_col]
    for j in range(1,32):
        curr1_col=curr1 + '_F' + str(j) + '_MA22'
        curr2_col=curr2 + '_F' + str(j) + '_MA22'    
        new_col=curr1 + '_' + curr2 + '_F' + str(j) + '_ratio'
        try:
            df_out[new_col]=df_in[curr1_col]/df_in[curr2_col]
        except:
            continue
    return df_out

## All the predictive analysis uses this monthly data set of the moving averages of the factors
## You can start with this

In [24]:
btc_eth_xrp_model=pd.read_excel('btc_eth_xrp_model.xlsx')
btc_eth_xrp_model_keep=btc_eth_xrp_model

In [25]:
def factor_lag(df_in,first_col,last_col):
    df_out=df_in.iloc[:,0:first_col]
    j=first_col
    for j in range(first_col,last_col):
        col_name_old=df_in.columns[j]
        col_name_new='L_'+col_name_old
        df_out[col_name_new]=df_in.iloc[:,j].shift(1)
        j=j+1
    return df_out

In [26]:
btc_eth_xrp_model_lags=factor_lag(btc_eth_xrp_model,9,93)
btc_eth_xrp_model_lags_keep=btc_eth_xrp_model_lags.dropna()
btc_eth_xrp_model_lags=btc_eth_xrp_model_lags_keep
btc_eth_xrp_price_clean=btc_eth_xrp_model_lags[['date','BTC_USD','BTC_Ret','ETH_USD','ETH_Ret','BTC_ETH_Ret','XRP_USD','XRP_Ret','BTC_XRP_Ret']].copy()
btc_eth_xrp_model_lags.to_csv('btc_eth_xrp_model_lags.csv',index=False)
btc_eth_xrp_model_lags.head()

,date,BTC_USD,BTC_Ret,ETH_USD,ETH_Ret,BTC_ETH_Ret,XRP_USD,XRP_Ret,BTC_XRP_Ret,L_BTC_F1_MA22,...,L_XRP_F22_MA22,L_XRP_F23_MA22,L_XRP_F24_MA22,L_XRP_F25_MA22,L_XRP_F26_MA22,L_XRP_F27_MA22,L_XRP_F28_MA22,L_XRP_F29_MA22,L_XRP_F30_MA22,L_XRP_F31_MA22
1,2015-10-31,314.119508,0.326855,0.889343,0.246199,0.080657,0.004631,-0.173314,0.500169,313073.136364,...,11534.500000,1.116494e+08,7.386826e+05,11886.352777,78.630943,1100.000000,7.250696,1.175701e+08,7.795747e+05,0.029242
2,2015-11-30,377.389840,0.201421,0.880000,-0.010505,0.211926,0.004204,-0.092315,0.293737,350885.363636,...,7400.954545,1.151984e+08,5.508404e+05,24708.658755,122.547418,775.609818,3.740480,1.207873e+08,5.763011e+05,0.051605
3,2015-12-31,429.677153,0.138550,0.948840,0.078227,0.060323,0.005955,0.416621,-0.278071,383127.136364,...,7896.590909,5.017505e+08,2.143587e+06,56445.962706,241.181455,1175.003091,5.039077,5.020341e+08,2.144797e+06,0.039679
4,2016-01-31,366.341688,-0.147402,2.242672,1.363593,-1.510996,0.006358,0.067630,-0.215032,456927.772727,...,14032.409091,3.548443e+08,2.827085e+06,21641.875393,151.513749,1255.822273,8.226846,3.555645e+08,2.832146e+06,0.080208
5,2016-02-29,437.775873,0.194993,6.333253,1.823977,-1.628984,0.008018,0.261072,-0.066079,488123.454545,...,15790.000000,1.871757e+08,1.093580e+06,17802.197137,97.419745,1106.819091,5.986345,1.883054e+08,1.099983e+06,0.042615


## Create model string

In [27]:
def model_string(curr,nfactors):
        lag_model_string= curr + '_Ret ~'
        for j in range(1,nfactors+1):
            if j>1:
                lag_model_string = lag_model_string + ' + L_' + curr +'_F' +str(j) + '_MA22'
            else:
                lag_model_string = lag_model_string + ' L_' + curr + '_F' +str(j) + '_MA22'
        j=j+1
        return lag_model_string

In [28]:
lm_string=model_string('BTC',31)
print(lm_string)

BTC_Ret ~ L_BTC_F1_MA22 + L_BTC_F2_MA22 + L_BTC_F3_MA22 + L_BTC_F4_MA22 + L_BTC_F5_MA22 + L_BTC_F6_MA22 + L_BTC_F7_MA22 + L_BTC_F8_MA22 + L_BTC_F9_MA22 + L_BTC_F10_MA22 + L_BTC_F11_MA22 + L_BTC_F12_MA22 + L_BTC_F13_MA22 + L_BTC_F14_MA22 + L_BTC_F15_MA22 + L_BTC_F16_MA22 + L_BTC_F17_MA22 + L_BTC_F18_MA22 + L_BTC_F19_MA22 + L_BTC_F20_MA22 + L_BTC_F21_MA22 + L_BTC_F22_MA22 + L_BTC_F23_MA22 + L_BTC_F24_MA22 + L_BTC_F25_MA22 + L_BTC_F26_MA22 + L_BTC_F27_MA22 + L_BTC_F28_MA22 + L_BTC_F29_MA22 + L_BTC_F30_MA22 + L_BTC_F31_MA22


In [30]:
lag_model=smf.ols(lm_string, data = btc_eth_xrp_model_lags).fit().summary()
print(lag_model)

                            OLS Regression Results                            
Dep. Variable:                BTC_Ret   R-squared:                       0.502
Model:                            OLS   Adj. R-squared:                  0.234
Method:                 Least Squares   F-statistic:                     1.876
Date:                Sat, 30 Jan 2021   Prob (F-statistic):             0.0403
Time:                        16:49:05   Log-Likelihood:                 23.709
No. Observations:                  64   AIC:                            -1.419
Df Residuals:                      41   BIC:                             48.24
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -0.6077      0.659     -0.

In [31]:
lm_string=model_string('ETH',31)
print(lm_string)

ETH_Ret ~ L_ETH_F1_MA22 + L_ETH_F2_MA22 + L_ETH_F3_MA22 + L_ETH_F4_MA22 + L_ETH_F5_MA22 + L_ETH_F6_MA22 + L_ETH_F7_MA22 + L_ETH_F8_MA22 + L_ETH_F9_MA22 + L_ETH_F10_MA22 + L_ETH_F11_MA22 + L_ETH_F12_MA22 + L_ETH_F13_MA22 + L_ETH_F14_MA22 + L_ETH_F15_MA22 + L_ETH_F16_MA22 + L_ETH_F17_MA22 + L_ETH_F18_MA22 + L_ETH_F19_MA22 + L_ETH_F20_MA22 + L_ETH_F21_MA22 + L_ETH_F22_MA22 + L_ETH_F23_MA22 + L_ETH_F24_MA22 + L_ETH_F25_MA22 + L_ETH_F26_MA22 + L_ETH_F27_MA22 + L_ETH_F28_MA22 + L_ETH_F29_MA22 + L_ETH_F30_MA22 + L_ETH_F31_MA22


In [32]:
lag_model=smf.ols(lm_string, data = btc_eth_xrp_model_lags).fit().summary()
print(lag_model)

                            OLS Regression Results                            
Dep. Variable:                ETH_Ret   R-squared:                       0.432
Model:                            OLS   Adj. R-squared:                  0.187
Method:                 Least Squares   F-statistic:                     1.765
Date:                Sat, 30 Jan 2021   Prob (F-statistic):             0.0607
Time:                        16:49:12   Log-Likelihood:                -32.492
No. Observations:                  64   AIC:                             105.0
Df Residuals:                      44   BIC:                             148.2
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept        2.94e-06    7.4e-06      0.

# Cross Section

## Relative factors

In [33]:
def relative_factors(df_in,first_col,curr1,curr2):
    df_out=df_in.iloc[:,0:first_col]
    for j in range(1,32):
        curr1_col=curr1 + '_F' + str(j) + '_MA22'
        curr2_col=curr2 + '_F' + str(j) + '_MA22'    
        new_col=curr1 + '_' + curr2 + '_F' + str(j) + '_ratio'
        try:
            df_out[new_col]=df_in[curr1_col]/df_in[curr2_col]
        except:
            continue
    return df_out

In [34]:
btc_eth_factor_ratios=relative_factors(btc_eth_xrp_model,6,'BTC','ETH')
btc_eth_factor_ratios.to_csv('btc_eth_factor_ratios.csv',index=False)
btc_eth_factor_ratios.head()

,date,BTC_USD,BTC_Ret,ETH_USD,ETH_Ret,BTC_ETH_Ret,BTC_ETH_F1_ratio,BTC_ETH_F2_ratio,BTC_ETH_F3_ratio,BTC_ETH_F4_ratio,...,BTC_ETH_F22_ratio,BTC_ETH_F23_ratio,BTC_ETH_F24_ratio,BTC_ETH_F25_ratio,BTC_ETH_F26_ratio,BTC_ETH_F27_ratio,BTC_ETH_F28_ratio,BTC_ETH_F29_ratio,BTC_ETH_F30_ratio,BTC_ETH_F31_ratio
0,2015-09-30,236.739848,0.026227,0.713644,-0.453778,0.480006,185.529819,0.029987,22.539703,750.941299,...,58.875540,0.913560,238.035190,0.042032,11.071263,0.011082,2.944359,2.491528,658.721151,0.236763
1,2015-10-31,314.119508,0.326855,0.889343,0.246199,0.080657,207.289957,0.029584,20.719704,703.078653,...,55.231016,0.929454,355.535684,0.046257,18.159671,0.012433,5.063084,2.530755,980.963708,0.144433
2,2015-11-30,377.389840,0.201421,0.880000,-0.010505,0.211926,197.891602,0.031100,20.324314,656.470446,...,56.401832,0.912583,335.049546,0.068696,25.080056,0.012616,4.672961,3.824531,1397.100442,0.415484
3,2015-12-31,429.677153,0.138550,0.948840,0.078227,0.060323,197.889897,0.032534,16.361776,507.440786,...,46.567979,1.441231,702.564236,0.201264,98.537622,0.037195,18.461064,9.858799,4814.863903,0.680464
4,2016-01-31,366.341688,-0.147402,2.242672,1.363593,-1.510996,139.014304,0.031607,19.312611,617.857093,...,42.442628,0.579069,121.018817,0.210480,45.708963,0.012129,2.859560,9.294581,1932.480748,0.675925


In [35]:
btc_eth_factor_ratios_lags=factor_lag(btc_eth_factor_ratios,6,37)
# btc_eth_factor_ratios_lags['BTC_ETH_Ret']=btc_eth_factor_ratios_lags['BTC_Ret']-btc_eth_factor_ratios_lags['ETH_Ret']
btc_eth_factor_ratios_lags_keep=btc_eth_factor_ratios_lags.dropna()
btc_eth_factor_ratios_lags=btc_eth_factor_ratios_lags_keep
btc_eth_factor_ratios_lags.to_csv('btc_eth_factor_ratios_lags.csv',index=False)
btc_eth_factor_ratios_lags.head()

,date,BTC_USD,BTC_Ret,ETH_USD,ETH_Ret,BTC_ETH_Ret,L_BTC_ETH_F1_ratio,L_BTC_ETH_F2_ratio,L_BTC_ETH_F3_ratio,L_BTC_ETH_F4_ratio,...,L_BTC_ETH_F22_ratio,L_BTC_ETH_F23_ratio,L_BTC_ETH_F24_ratio,L_BTC_ETH_F25_ratio,L_BTC_ETH_F26_ratio,L_BTC_ETH_F27_ratio,L_BTC_ETH_F28_ratio,L_BTC_ETH_F29_ratio,L_BTC_ETH_F30_ratio,L_BTC_ETH_F31_ratio
1,2015-10-31,314.119508,0.326855,0.889343,0.246199,0.080657,185.529819,0.029987,22.539703,750.941299,...,58.875540,0.913560,238.035190,0.042032,11.071263,0.011082,2.944359,2.491528,658.721151,0.236763
2,2015-11-30,377.389840,0.201421,0.880000,-0.010505,0.211926,207.289957,0.029584,20.719704,703.078653,...,55.231016,0.929454,355.535684,0.046257,18.159671,0.012433,5.063084,2.530755,980.963708,0.144433
3,2015-12-31,429.677153,0.138550,0.948840,0.078227,0.060323,197.891602,0.031100,20.324314,656.470446,...,56.401832,0.912583,335.049546,0.068696,25.080056,0.012616,4.672961,3.824531,1397.100442,0.415484
4,2016-01-31,366.341688,-0.147402,2.242672,1.363593,-1.510996,197.889897,0.032534,16.361776,507.440786,...,46.567979,1.441231,702.564236,0.201264,98.537622,0.037195,18.461064,9.858799,4814.863903,0.680464
5,2016-02-29,437.775873,0.194993,6.333253,1.823977,-1.628984,139.014304,0.031607,19.312611,617.857093,...,42.442628,0.579069,121.018817,0.210480,45.708963,0.012129,2.859560,9.294581,1932.480748,0.675925


In [36]:
btc_xrp_factor_ratios=relative_factors(btc_eth_xrp_model,9,'BTC','XRP')
btc_xrp_factor_ratios.to_csv('btc_xrp_factor_ratios.csv',index=False)
btc_xrp_factor_ratios.head()

,date,BTC_USD,BTC_Ret,ETH_USD,ETH_Ret,BTC_ETH_Ret,XRP_USD,XRP_Ret,BTC_XRP_Ret,BTC_XRP_F1_ratio,...,BTC_XRP_F22_ratio,BTC_XRP_F23_ratio,BTC_XRP_F24_ratio,BTC_XRP_F25_ratio,BTC_XRP_F26_ratio,BTC_XRP_F27_ratio,BTC_XRP_F28_ratio,BTC_XRP_F29_ratio,BTC_XRP_F30_ratio,BTC_XRP_F31_ratio
0,2015-09-30,236.739848,0.026227,0.713644,-0.453778,0.480006,0.005602,-0.271297,0.297524,201.056981,...,29.655051,0.002947,104.234952,0.000271,9.573335,0.000020,0.707029,0.009206,324.751935,0.869849
1,2015-10-31,314.119508,0.326855,0.889343,0.246199,0.080657,0.004631,-0.173314,0.500169,241.792833,...,47.222195,0.003807,222.555293,0.000172,9.646062,0.000034,1.936810,0.012301,720.608832,0.275456
2,2015-11-30,377.389840,0.201421,0.880000,-0.010505,0.211926,0.004204,-0.092315,0.293737,362.092834,...,48.973251,0.000900,71.742416,0.000095,7.559416,0.000024,1.918428,0.004136,328.320322,1.210252
3,2015-12-31,429.677153,0.138550,0.948840,0.078227,0.060323,0.005955,0.416621,-0.278071,309.105224,...,33.611403,0.001177,65.348875,0.000322,20.401526,0.000035,2.390783,0.009431,525.969617,0.405446
4,2016-01-31,366.341688,-0.147402,2.242672,1.363593,-1.510996,0.006358,0.067630,-0.215032,395.372630,...,31.365735,0.002502,169.691314,0.001036,74.564408,0.000015,1.106732,0.050511,3407.800811,0.901341


In [37]:
btc_eth_xrp_model_lags=factor_lag(btc_eth_xrp_model,9,93)
btc_eth_xrp_model_lags_keep=btc_eth_xrp_model_lags.dropna()
btc_eth_xrp_model_lags=btc_eth_xrp_model_lags_keep
btc_eth_xrp_model_lags.head()

,date,BTC_USD,BTC_Ret,ETH_USD,ETH_Ret,BTC_ETH_Ret,XRP_USD,XRP_Ret,BTC_XRP_Ret,L_BTC_F1_MA22,...,L_XRP_F22_MA22,L_XRP_F23_MA22,L_XRP_F24_MA22,L_XRP_F25_MA22,L_XRP_F26_MA22,L_XRP_F27_MA22,L_XRP_F28_MA22,L_XRP_F29_MA22,L_XRP_F30_MA22,L_XRP_F31_MA22
1,2015-10-31,314.119508,0.326855,0.889343,0.246199,0.080657,0.004631,-0.173314,0.500169,313073.136364,...,11534.500000,1.116494e+08,7.386826e+05,11886.352777,78.630943,1100.000000,7.250696,1.175701e+08,7.795747e+05,0.029242
2,2015-11-30,377.389840,0.201421,0.880000,-0.010505,0.211926,0.004204,-0.092315,0.293737,350885.363636,...,7400.954545,1.151984e+08,5.508404e+05,24708.658755,122.547418,775.609818,3.740480,1.207873e+08,5.763011e+05,0.051605
3,2015-12-31,429.677153,0.138550,0.948840,0.078227,0.060323,0.005955,0.416621,-0.278071,383127.136364,...,7896.590909,5.017505e+08,2.143587e+06,56445.962706,241.181455,1175.003091,5.039077,5.020341e+08,2.144797e+06,0.039679
4,2016-01-31,366.341688,-0.147402,2.242672,1.363593,-1.510996,0.006358,0.067630,-0.215032,456927.772727,...,14032.409091,3.548443e+08,2.827085e+06,21641.875393,151.513749,1255.822273,8.226846,3.555645e+08,2.832146e+06,0.080208
5,2016-02-29,437.775873,0.194993,6.333253,1.823977,-1.628984,0.008018,0.261072,-0.066079,488123.454545,...,15790.000000,1.871757e+08,1.093580e+06,17802.197137,97.419745,1106.819091,5.986345,1.883054e+08,1.099983e+06,0.042615


In [38]:
#btc_xrp_factor_ratios_lags=factor_lag(btc_xrp_factor_ratios,8,30)
#btc_xrp_factor_ratios_lags['BTC_XRP_Ret']=btc_xrp_factor_ratios_lags['BTC_Ret']-btc_xrp_factor_ratios_lags['XRP_Ret']
btc_xrp_factor_ratios_lags=factor_lag(btc_xrp_factor_ratios,9,30)
btc_xrp_factor_ratios_lags_keep=btc_xrp_factor_ratios_lags.dropna()
btc_xrp_factor_ratios_lags=btc_xrp_factor_ratios_lags_keep
btc_xrp_factor_ratios_lags.to_csv('btc_xrp_factor_ratios_lags.csv',index=False)
btc_xrp_factor_ratios_lags.head()

,date,BTC_USD,BTC_Ret,ETH_USD,ETH_Ret,BTC_ETH_Ret,XRP_USD,XRP_Ret,BTC_XRP_Ret,L_BTC_XRP_F1_ratio,...,L_BTC_XRP_F21_ratio,L_BTC_XRP_F22_ratio,L_BTC_XRP_F23_ratio,L_BTC_XRP_F24_ratio,L_BTC_XRP_F25_ratio,L_BTC_XRP_F26_ratio,L_BTC_XRP_F27_ratio,L_BTC_XRP_F28_ratio,L_BTC_XRP_F29_ratio,L_BTC_XRP_F30_ratio
1,2015-10-31,314.119508,0.326855,0.889343,0.246199,0.080657,0.004631,-0.173314,0.500169,201.056981,...,0.554464,29.655051,0.002947,104.234952,0.000271,9.573335,0.000020,0.707029,0.009206,324.751935
2,2015-11-30,377.389840,0.201421,0.880000,-0.010505,0.211926,0.004204,-0.092315,0.293737,241.792833,...,0.699529,47.222195,0.003807,222.555293,0.000172,9.646062,0.000034,1.936810,0.012301,720.608832
3,2015-12-31,429.677153,0.138550,0.948840,0.078227,0.060323,0.005955,0.416621,-0.278071,362.092834,...,0.586381,48.973251,0.000900,71.742416,0.000095,7.559416,0.000024,1.918428,0.004136,328.320322
4,2016-01-31,366.341688,-0.147402,2.242672,1.363593,-1.510996,0.006358,0.067630,-0.215032,309.105224,...,0.544663,33.611403,0.001177,65.348875,0.000322,20.401526,0.000035,2.390783,0.009431,525.969617
5,2016-02-29,437.775873,0.194993,6.333253,1.823977,-1.628984,0.008018,0.261072,-0.066079,395.372630,...,0.391988,31.365735,0.002502,169.691314,0.001036,74.564408,0.000015,1.106732,0.050511,3407.800811


In [39]:
def model_spec(df_in,first_col,last_col,curr,curr2):
        lag_model_string= curr + '_' + curr2 + '_Ret ~ '
        for j in range(first_col,last_col):
            if j>first_col:
                lag_model_string = lag_model_string + ' + ' + df_in.columns[j]
            else:
                lag_model_string = lag_model_string  + df_in.columns[j]
        j=j+1
        return lag_model_string

In [40]:
btc_eth_ratio_lag_model=model_spec(btc_eth_factor_ratios_lags,6,37,'BTC','ETH')
print(btc_eth_ratio_lag_model)

BTC_ETH_Ret ~ L_BTC_ETH_F1_ratio + L_BTC_ETH_F2_ratio + L_BTC_ETH_F3_ratio + L_BTC_ETH_F4_ratio + L_BTC_ETH_F5_ratio + L_BTC_ETH_F6_ratio + L_BTC_ETH_F7_ratio + L_BTC_ETH_F8_ratio + L_BTC_ETH_F9_ratio + L_BTC_ETH_F10_ratio + L_BTC_ETH_F11_ratio + L_BTC_ETH_F12_ratio + L_BTC_ETH_F13_ratio + L_BTC_ETH_F14_ratio + L_BTC_ETH_F15_ratio + L_BTC_ETH_F16_ratio + L_BTC_ETH_F17_ratio + L_BTC_ETH_F18_ratio + L_BTC_ETH_F19_ratio + L_BTC_ETH_F20_ratio + L_BTC_ETH_F21_ratio + L_BTC_ETH_F22_ratio + L_BTC_ETH_F23_ratio + L_BTC_ETH_F24_ratio + L_BTC_ETH_F25_ratio + L_BTC_ETH_F26_ratio + L_BTC_ETH_F27_ratio + L_BTC_ETH_F28_ratio + L_BTC_ETH_F29_ratio + L_BTC_ETH_F30_ratio + L_BTC_ETH_F31_ratio


In [61]:
btc_xrp_ratio_lag_model=model_spec(btc_xrp_factor_ratios_lags,9,30,'BTC','XRP')
print(btc_xrp_ratio_lag_model)

BTC_XRP_Ret ~ L_BTC_XRP_F1_ratio + L_BTC_XRP_F2_ratio + L_BTC_XRP_F5_ratio + L_BTC_XRP_F7_ratio + L_BTC_XRP_F8_ratio + L_BTC_XRP_F9_ratio + L_BTC_XRP_F10_ratio + L_BTC_XRP_F11_ratio + L_BTC_XRP_F12_ratio + L_BTC_XRP_F19_ratio + L_BTC_XRP_F20_ratio + L_BTC_XRP_F21_ratio + L_BTC_XRP_F22_ratio + L_BTC_XRP_F23_ratio + L_BTC_XRP_F24_ratio + L_BTC_XRP_F25_ratio + L_BTC_XRP_F26_ratio + L_BTC_XRP_F27_ratio + L_BTC_XRP_F28_ratio + L_BTC_XRP_F29_ratio + L_BTC_XRP_F30_ratio


In [43]:
model=smf.ols(btc_eth_ratio_lag_model, data = btc_eth_factor_ratios_lags).fit().summary()
fit=smf.ols(btc_eth_ratio_lag_model, data = btc_eth_factor_ratios_lags).fit()
btc_eth_factor_model_pred=pd.DataFrame(columns=['date','BTC_ETH_Ret','Fitted Lag OLS Factor Ratios'])
btc_eth_factor_model_pred['date']=btc_eth_factor_ratios_lags['date']
btc_eth_factor_model_pred['BTC_ETH_Ret']=btc_eth_factor_ratios_lags['BTC_ETH_Ret']
btc_eth_factor_model_pred['Fitted Lag OLS Factor Ratios']=fit.fittedvalues
print(btc_eth_factor_model_pred.head())
print(model)

         date  BTC_ETH_Ret  Fitted Lag OLS Factor Ratios
1  2015-10-31     0.080657                      0.049951
2  2015-11-30     0.211926                      0.251851
3  2015-12-31     0.060323                     -0.027341
4  2016-01-31    -1.510996                     -1.481920
5  2016-02-29    -1.628984                     -1.680866
                            OLS Regression Results                            
Dep. Variable:            BTC_ETH_Ret   R-squared:                       0.806
Model:                            OLS   Adj. R-squared:                  0.640
Method:                 Least Squares   F-statistic:                     4.866
Date:                Sat, 30 Jan 2021   Prob (F-statistic):           9.34e-06
Time:                        16:53:16   Log-Likelihood:                 7.8635
No. Observations:                  64   AIC:                             44.27
Df Residuals:                      34   BIC:                             109.0
Df Model:                 

In [62]:
model=smf.ols(btc_xrp_ratio_lag_model, data = btc_xrp_factor_ratios_lags).fit().summary()
print(model)

                            OLS Regression Results                            
Dep. Variable:            BTC_XRP_Ret   R-squared:                       0.419
Model:                            OLS   Adj. R-squared:                  0.128
Method:                 Least Squares   F-statistic:                     1.442
Date:                Sat, 30 Jan 2021   Prob (F-statistic):              0.154
Time:                        17:11:56   Log-Likelihood:                -77.442
No. Observations:                  64   AIC:                             198.9
Df Residuals:                      42   BIC:                             246.4
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              -8.0288    

## Good data one from each category

In [46]:
# good_data_model='BTC_ETH_Ret ~ L_BTC_ETH_F1_ratio + L_BTC_ETH_F2_ratio + L_BTC_ETH_F6_ratio + L_BTC_ETH_F8_ratio'
good_data_model='BTC_ETH_Ret ~ L_BTC_ETH_F6_ratio + L_BTC_ETH_F7_ratio'
good_data_model=good_data_model + ' + L_BTC_ETH_F8_ratio + L_BTC_ETH_F9_ratio + L_BTC_ETH_F10_ratio + L_BTC_ETH_F11_ratio'
good_data_model=good_data_model + ' + L_BTC_ETH_F12_ratio + L_BTC_ETH_F13_ratio + L_BTC_ETH_F15_ratio'
good_data_model=good_data_model + ' + L_BTC_ETH_F20_ratio + L_BTC_ETH_F21_ratio + L_BTC_ETH_F22_ratio'
good_data_model=good_data_model + ' + L_BTC_ETH_F25_ratio + L_BTC_ETH_F26_ratio + L_BTC_ETH_F27_ratio'
good_data_model=good_data_model + ' + L_BTC_ETH_F28_ratio + L_BTC_ETH_F29_ratio + L_BTC_ETH_F30_ratio + L_BTC_ETH_F31_ratio'
print(good_data_model)

BTC_ETH_Ret ~ L_BTC_ETH_F6_ratio + L_BTC_ETH_F7_ratio + L_BTC_ETH_F8_ratio + L_BTC_ETH_F9_ratio + L_BTC_ETH_F10_ratio + L_BTC_ETH_F11_ratio + L_BTC_ETH_F12_ratio + L_BTC_ETH_F13_ratio + L_BTC_ETH_F15_ratio + L_BTC_ETH_F20_ratio + L_BTC_ETH_F21_ratio + L_BTC_ETH_F22_ratio + L_BTC_ETH_F25_ratio + L_BTC_ETH_F26_ratio + L_BTC_ETH_F27_ratio + L_BTC_ETH_F28_ratio + L_BTC_ETH_F29_ratio + L_BTC_ETH_F30_ratio + L_BTC_ETH_F31_ratio


In [47]:
gd_model=smf.ols(good_data_model, data = btc_eth_factor_ratios_lags).fit().summary()
print(gd_model)

                            OLS Regression Results                            
Dep. Variable:            BTC_ETH_Ret   R-squared:                       0.688
Model:                            OLS   Adj. R-squared:                  0.553
Method:                 Least Squares   F-statistic:                     5.102
Date:                Sat, 30 Jan 2021   Prob (F-statistic):           4.11e-06
Time:                        16:58:22   Log-Likelihood:                -7.3340
No. Observations:                  64   AIC:                             54.67
Df Residuals:                      44   BIC:                             97.85
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              -4.1294    

## PCA

In [48]:
btc_eth_xrp_model_clean=btc_eth_xrp_model.dropna()
btc_eth_xrp_model_clean.head()

,date,BTC_USD,BTC_Ret,ETH_USD,ETH_Ret,BTC_ETH_Ret,XRP_USD,XRP_Ret,BTC_XRP_Ret,BTC_F1_MA22,...,XRP_F22_MA22,XRP_F23_MA22,XRP_F24_MA22,XRP_F25_MA22,XRP_F26_MA22,XRP_F27_MA22,XRP_F28_MA22,XRP_F29_MA22,XRP_F30_MA22,XRP_F31_MA22
0,2015-09-30,236.739848,0.026227,0.713644,-0.453778,0.480006,0.005602,-0.271297,0.297524,313073.136364,...,11534.500000,1.116494e+08,7.386826e+05,11886.352777,78.630943,1100.000000,7.250696,1.175701e+08,7.795747e+05,0.029242
1,2015-10-31,314.119508,0.326855,0.889343,0.246199,0.080657,0.004631,-0.173314,0.500169,350885.363636,...,7400.954545,1.151984e+08,5.508404e+05,24708.658755,122.547418,775.609818,3.740480,1.207873e+08,5.763011e+05,0.051605
2,2015-11-30,377.389840,0.201421,0.880000,-0.010505,0.211926,0.004204,-0.092315,0.293737,383127.136364,...,7896.590909,5.017505e+08,2.143587e+06,56445.962706,241.181455,1175.003091,5.039077,5.020341e+08,2.144797e+06,0.039679
3,2015-12-31,429.677153,0.138550,0.948840,0.078227,0.060323,0.005955,0.416621,-0.278071,456927.772727,...,14032.409091,3.548443e+08,2.827085e+06,21641.875393,151.513749,1255.822273,8.226846,3.555645e+08,2.832146e+06,0.080208
4,2016-01-31,366.341688,-0.147402,2.242672,1.363593,-1.510996,0.006358,0.067630,-0.215032,488123.454545,...,15790.000000,1.871757e+08,1.093580e+06,17802.197137,97.419745,1106.819091,5.986345,1.883054e+08,1.099983e+06,0.042615


In [49]:
btc_eth_xrp_model_clean_df2=btc_eth_xrp_model_clean.set_index('date')
btc_eth_xrp_model_clean_df3=btc_eth_xrp_model_clean_df2.pct_change()
btc_eth_xrp_model_clean_df4=btc_eth_xrp_model_clean_df3.dropna()

In [50]:
normalize = lambda x: (x-x.mean())/x.std()

In [51]:
from sklearn.decomposition import PCA

In [52]:
factor_ma_norm = btc_eth_xrp_model_clean_df4.apply(normalize)
factor_ma_norm.head()

,BTC_USD,BTC_Ret,ETH_USD,ETH_Ret,BTC_ETH_Ret,XRP_USD,XRP_Ret,BTC_XRP_Ret,BTC_F1_MA22,BTC_F2_MA22,...,XRP_F22_MA22,XRP_F23_MA22,XRP_F24_MA22,XRP_F25_MA22,XRP_F26_MA22,XRP_F27_MA22,XRP_F28_MA22,XRP_F29_MA22,XRP_F30_MA22,XRP_F31_MA22
date,,,,,,,,,,,,,,,,,,,,,
2015-10-31,0.922722,0.335810,0.056488,-0.169689,-0.030414,-0.376792,0.133421,0.380163,1.060414,-0.013658,...,-1.186121,-0.274101,-0.487353,0.104080,-0.046377,-0.353308,-0.532229,-0.278102,-0.494567,0.830923
2015-11-30,0.396829,0.133089,-0.420775,-0.097934,0.301654,-0.305343,0.132740,0.139860,0.740867,0.682960,...,-0.062503,3.343183,1.342538,0.154080,0.067841,0.083139,-0.060977,3.180346,1.271110,-0.560721
2015-12-31,0.133235,0.134315,-0.255804,-1.160736,-0.014670,0.143589,0.100356,-0.197132,1.855109,0.626849,...,1.813267,-0.627341,-0.153953,-0.307968,-0.305825,-0.157309,0.100834,-0.630820,-0.150466,1.189634
2016-01-31,-1.065646,0.104338,2.133944,2.410344,-3.435042,-0.164256,0.130363,0.180728,0.479681,-0.524532,...,0.091465,-0.822940,-0.696115,-0.201228,-0.301708,-0.258322,-0.412173,-0.828283,-0.702286,-0.892747
2016-02-29,0.369879,0.099906,2.989888,0.100200,0.092457,0.006380,0.154097,0.078352,0.035224,-0.814687,...,0.927675,-0.589560,-0.334920,-0.317773,-0.342277,-0.395785,-0.302417,-0.591729,-0.334212,0.234284


In [53]:
pca = PCA(n_components=10)
pca.fit(factor_ma_norm)

PCA(n_components=10)

In [54]:
factor_ma_pca = pca.transform(factor_ma_norm)
factor_ma_pca_df = pd.DataFrame(data=factor_ma_pca,columns = ['PC1', 'PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10'])

In [55]:
factor_ma_pca_df['date']=btc_eth_xrp_model_clean_df4.index
factor_ma_pca_df.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,date
0,-0.134991,-1.243999,-0.599841,-0.976391,2.179889,1.186851,-0.472959,-0.826639,-0.349770,-0.263577,2015-10-31
1,1.896900,0.894022,-0.460541,-3.042006,0.594566,2.275901,1.501062,1.802648,-0.974803,0.010029,2015-11-30
2,4.175722,3.484283,-3.096815,-0.079452,1.434198,0.545407,-3.162232,-4.631492,-1.015601,0.545323,2015-12-31
3,2.410478,-5.067246,8.123560,-3.553636,6.891478,2.301473,-2.031934,-2.373296,-1.930614,1.624414,2016-01-31
4,5.448619,0.432612,4.350659,-0.977450,-5.071525,-3.406947,-3.826848,-2.093162,5.498779,-4.745163,2016-02-29


In [56]:
pca_model_df=pd.merge(btc_eth_xrp_price_clean,factor_ma_pca_df, on='date', how='outer')
pca_model_df.head()

,date,BTC_USD,BTC_Ret,ETH_USD,ETH_Ret,BTC_ETH_Ret,XRP_USD,XRP_Ret,BTC_XRP_Ret,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10
0,2015-10-31,314.119508,0.326855,0.889343,0.246199,0.080657,0.004631,-0.173314,0.500169,-0.134991,-1.243999,-0.599841,-0.976391,2.179889,1.186851,-0.472959,-0.826639,-0.349770,-0.263577
1,2015-11-30,377.389840,0.201421,0.880000,-0.010505,0.211926,0.004204,-0.092315,0.293737,1.896900,0.894022,-0.460541,-3.042006,0.594566,2.275901,1.501062,1.802648,-0.974803,0.010029
2,2015-12-31,429.677153,0.138550,0.948840,0.078227,0.060323,0.005955,0.416621,-0.278071,4.175722,3.484283,-3.096815,-0.079452,1.434198,0.545407,-3.162232,-4.631492,-1.015601,0.545323
3,2016-01-31,366.341688,-0.147402,2.242672,1.363593,-1.510996,0.006358,0.067630,-0.215032,2.410478,-5.067246,8.123560,-3.553636,6.891478,2.301473,-2.031934,-2.373296,-1.930614,1.624414
4,2016-02-29,437.775873,0.194993,6.333253,1.823977,-1.628984,0.008018,0.261072,-0.066079,5.448619,0.432612,4.350659,-0.977450,-5.071525,-3.406947,-3.826848,-2.093162,5.498779,-4.745163


In [58]:
model=smf.ols('BTC_ETH_Ret ~ PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10', data = pca_model_df).fit().summary()
print(model)

                            OLS Regression Results                            
Dep. Variable:            BTC_ETH_Ret   R-squared:                       0.801
Model:                            OLS   Adj. R-squared:                  0.764
Method:                 Least Squares   F-statistic:                     21.37
Date:                Sat, 30 Jan 2021   Prob (F-statistic):           3.17e-15
Time:                        16:59:54   Log-Likelihood:                 7.1258
No. Observations:                  64   AIC:                             7.748
Df Residuals:                      53   BIC:                             31.50
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.1090      0.030     -3.667      0.0

In [59]:
pca_model_df_lags=factor_lag(pca_model_df,9,19)
pca_model_df_lags.head()

,date,BTC_USD,BTC_Ret,ETH_USD,ETH_Ret,BTC_ETH_Ret,XRP_USD,XRP_Ret,BTC_XRP_Ret,L_PC1,L_PC2,L_PC3,L_PC4,L_PC5,L_PC6,L_PC7,L_PC8,L_PC9,L_PC10
0,2015-10-31,314.119508,0.326855,0.889343,0.246199,0.080657,0.004631,-0.173314,0.500169,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-11-30,377.389840,0.201421,0.880000,-0.010505,0.211926,0.004204,-0.092315,0.293737,-0.134991,-1.243999,-0.599841,-0.976391,2.179889,1.186851,-0.472959,-0.826639,-0.349770,-0.263577
2,2015-12-31,429.677153,0.138550,0.948840,0.078227,0.060323,0.005955,0.416621,-0.278071,1.896900,0.894022,-0.460541,-3.042006,0.594566,2.275901,1.501062,1.802648,-0.974803,0.010029
3,2016-01-31,366.341688,-0.147402,2.242672,1.363593,-1.510996,0.006358,0.067630,-0.215032,4.175722,3.484283,-3.096815,-0.079452,1.434198,0.545407,-3.162232,-4.631492,-1.015601,0.545323
4,2016-02-29,437.775873,0.194993,6.333253,1.823977,-1.628984,0.008018,0.261072,-0.066079,2.410478,-5.067246,8.123560,-3.553636,6.891478,2.301473,-2.031934,-2.373296,-1.930614,1.624414


In [60]:
lag_model=smf.ols('BTC_ETH_Ret ~ L_PC1+L_PC2+L_PC3+L_PC4+L_PC5+L_PC6+L_PC7+L_PC8+L_PC9+L_PC10', data = pca_model_df_lags).fit().summary()
print(lag_model)

                            OLS Regression Results                            
Dep. Variable:            BTC_ETH_Ret   R-squared:                       0.370
Model:                            OLS   Adj. R-squared:                  0.249
Method:                 Least Squares   F-statistic:                     3.060
Date:                Sat, 30 Jan 2021   Prob (F-statistic):            0.00401
Time:                        17:00:02   Log-Likelihood:                -29.732
No. Observations:                  63   AIC:                             81.46
Df Residuals:                      52   BIC:                             105.0
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.1132      0.054     -2.104      0.0

## Kernel PCA

In [ ]:
from sklearn.decomposition import KernelPCA 
kpca = KernelPCA(kernel ='poly',n_components=10,degree=2) 
factor_ma_kpca = kpca.fit_transform(factor_ma_norm)
factor_ma_kpca_df = pd.DataFrame(data=factor_ma_kpca,columns = ['KPC1', 'KPC2','KPC3','KPC4','KPC5','KPC6','KPC7','KPC8','KPC9','KPC10'])
factor_ma_kpca_df['date']=btc_eth_xrp_model_clean_df4.index
kpca_model_df=pd.merge(btc_eth_xrp_price_clean,factor_ma_kpca_df, on='date', how='outer')
# model=smf.ols('BTC_ETH_Ret ~ KPC1+KPC2+KPC3+KPC4+KPC5+KPC6+KPC7+KPC8+KPC9+KPC10', data = kpca_model_df).fit().summary()
model=smf.ols('BTC_ETH_Ret ~ KPC1+KPC2+KPC3+KPC4+KPC5+KPC6+KPC7+KPC8+KPC9+KPC10', data = kpca_model_df).fit()
# print(model)
print('Cur. Model Adj. R2: ', model.rsquared_adj.round(6))
kpca_model_df_lags=factor_lag(kpca_model_df,9,19)
# lag_model=smf.ols('BTC_ETH_Ret ~ L_KPC1+L_KPC2+L_KPC3+L_KPC4+L_KPC5+L_KPC6+L_KPC7+L_KPC8+L_KPC9+L_KPC10', data = kpca_model_df_lags).fit().summary()
lag_model=smf.ols('BTC_ETH_Ret ~ L_KPC1+L_KPC2+L_KPC3+L_KPC4+L_KPC5+L_KPC6+L_KPC7+L_KPC8+L_KPC9+L_KPC10', data = kpca_model_df_lags).fit()
# print(lag_model)
print('Lag model Adj. R2: ', lag_model.rsquared_adj.round(5))

In [ ]:
btc_eth_factor_model_pred['Fitted Kernel PCA Lags']=lag_model.fittedvalues
# btc_eth_factor_model_pred.head(100)
btc_eth_factor_model_pred.iloc[35:53,:]

## Neural networks

In [ ]:
nn_model_df=factor_ma_norm
nn_model_df.head()
X=nn_model_df.iloc[:,8:92]

In [ ]:
nn_model_df=btc_eth_factor_ratios
X=nn_model_df.iloc[:,6:37]
nn_model_df.head()

In [ ]:
from sklearn.model_selection import train_test_split
y=nn_model_df['BTC_ETH_Ret']

In [ ]:
# Split the data up in train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
from numpy.random import seed
seed(5821)
from tensorflow import set_random_seed
set_random_seed(5822)
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
NN_model = keras.models.Sequential()
# This is for ratio model
NN_model.add(keras.layers.Flatten(input_shape=[31,]))
# This is for all currency MA factors
# NN_model.add(keras.layers.Flatten(input_shape=[83,]))
NN_model.add(keras.layers.Dense(16, activation="sigmoid"))
NN_model.add(keras.layers.Dense(16, activation="sigmoid"))
# NN_model.add(keras.layers.Dense(16, activation="sigmoid"))
NN_model.add(keras.layers.Dense(1, activation="sigmoid"))
# NN_model.summary()
NN_model.compile(loss='mse',optimizer='adam',metrics=['mean_squared_error'])   
NN_model.fit(X_train, y_train,epochs=20, batch_size=1, verbose=0)
y_pred = NN_model.predict(X_test)
btc_eth_all_models_pred['Fit NN Model']=y_pred[:,0]
nn_rmse=np.sqrt(mean_squared_error(btc_eth_all_models_pred['BTC_ETH_Ret'], btc_eth_all_models_pred['Fit NN Model']))
print(nn_rmse)

In [ ]:
# Import `Sequential` from `keras.models`
from keras.models import Sequential

# Import `Dense` from `keras.layers`
from keras.layers import Dense

# Initialize the constructor
NN_model = Sequential()

# Add an input layer 
NN_model.add(Dense(90, activation='tanh', input_shape=(83,)))

# Add one hidden layer 
NN_model.add(Dense(83, activation='tanh'))

# Add an output layer 
NN_model.add(Dense(1, activation='sigmoid'))
NN_model.summary()

In [ ]:
NN_model.compile(loss='mse',optimizer='adam',metrics=['accuracy'])   
NN_model.fit(X_train, y_train,epochs=20, batch_size=1, verbose=1)
y_pred = NN_model.predict(X_test)
btc_eth_all_models_pred['Fit NN Model']=y_pred[:,0]
nn_rmse=np.sqrt(mean_squared_error(btc_eth_all_models_pred['BTC_ETH_Ret'], btc_eth_all_models_pred['Fit NN Model']))
print(nn_rmse)

In [ ]:
y_pred = NN_model.predict(X_test)

In [ ]:
btc_eth_factor_model_pred['Fitted Kernel PCA Lags']=lag_model.fittedvalues
btc_eth_factor_model_pred['Fitted Kernel PCA Lags'].iloc[52]=0
btc_eth_all_models_pred=btc_eth_factor_model_pred.iloc[35:53,:].copy()
btc_eth_all_models_pred['Fit NN Model']=y_pred[:,0]
btc_eth_all_models_pred.head(50)

In [ ]:
y1=btc_eth_all_models_pred['BTC_ETH_Ret']
x=btc_eth_all_models_pred['date']
g=sns.scatterplot(x, y1,color='blue')
y2=btc_eth_all_models_pred['Fit NN Model']
g=sns.scatterplot(x, y2,color='red')
y3=btc_eth_all_models_pred['Fitted Lag OLS Factor Ratios']
g=sns.scatterplot(x, y3,color='green')
y4=btc_eth_all_models_pred['Fitted Kernel PCA Lags']
g=sns.scatterplot(x, y4,color='purple')
g.legend(('Data','NN Model','OLS','Kernel PCA'),loc='lower left', bbox_to_anchor=(1.25, 0.5), ncol=1)
plt.xticks(rotation=90)
plt.show()

In [ ]:
btc_eth_all_models_pred['Fit NN Model']=y_pred[:,0]

In [ ]:
from sklearn.metrics import mean_squared_error
nn_rmse=np.sqrt(mean_squared_error(btc_eth_all_models_pred['BTC_ETH_Ret'], btc_eth_all_models_pred['Fit NN Model']))
kernelPCA_rmse=np.sqrt(mean_squared_error(btc_eth_all_models_pred['BTC_ETH_Ret'], btc_eth_all_models_pred['Fitted Kernel PCA Lags']))
ols_rmse=np.sqrt(mean_squared_error(btc_eth_all_models_pred['BTC_ETH_Ret'], btc_eth_all_models_pred['Fitted Lag OLS Factor Ratios']))
print('OLS Model: ',ols_rmse.round(4),'Kernel PCA: ',kernelPCA_rmse.round(4),'NN Model: ',nn_rmse.round(4))